In [65]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interpn
from skimage import measure
import os
import tkinter as tk
from tkinter import filedialog
import numpy as np
from skimage.measure import label, regionprops
from scipy.spatial import cKDTree

In [66]:

def parse_input_args(voxel_grid, **kwargs):
    # 设置默认参数
    defaults = {
        'verbose': True,
        'padding_size': int(np.ceil(12 * voxel_grid['truncation'] / voxel_grid['interval'])),
        'min_area': int(np.ceil(voxel_grid['size'][0] / 20)),
        'max_division': 50,
        'scaleInitRatio': 0.1,
        'nanRange': 0.5 * voxel_grid['interval'],
        'w': 0.99,
        'tolerance': 1e-6,
        'relative_tolerance': 1e-4,
        'switch_tolerance': 1e-1,
        'maxSwitch': 2,
        'iter_min': 2,
        'maxOptiIter': 2,
        'maxIter': 15,
        'activeMultiplier': 3
    }

    # 使用从kwargs传入的参数来更新默认参数
    for key, value in kwargs.items():
        if key in defaults:
            defaults[key] = value

    return defaults

def idx3d_flatten(indices, grid):
    # 将3D索引平铺为1D索引
    return indices[:, 0] + grid['size'][0] * (indices[:, 1] - 1) + grid['size'][0] * grid['size'][1] * (indices[:, 2] - 1)

def idx2Coordinate(indices, grid):
    # 从网格索引转换为3D坐标
    x = grid['x'][indices[:, 0] - 1]
    y = grid['y'][indices[:, 1] - 1]
    z = grid['z'][indices[:, 2] - 1]
    return np.vstack([x, y, z]).T

def mps(sdf, voxel_grid, **kargs):
    # 这里假设parameters是一个包含算法超参数的字典
    parameters = parse_input_args(voxel_grid, **kargs)
    # 初始化一些变量
    num_division = 1
    x = []
    dratio = 3/5
    conn_ratio = [1, dratio, dratio**2, dratio**3, dratio**4,
                  dratio**5, dratio**6, dratio**7, dratio**8]

    # 初始化连接指针和区域数量
    conn_pointer = 1
    num_region = 1

    while num_division < parameters['max_division']:
        # 1-连接性行进
        if conn_pointer != 1 and num_region != 0:
            conn_pointer = 1

        # 设置连接阈值
        conn_threshold = conn_ratio[conn_pointer - 1] * np.min(sdf)
        if conn_threshold > -voxel_grid['truncation'] * 0.3:
            break

        # 将sdf重排成三维数组，用于连接性检查
        sdf3d_region = sdf.reshape((voxel_grid['size'][0], voxel_grid['size'][1], voxel_grid['size'][2]))

        # 连接性检查和初步划分
        labeled_region = label(sdf3d_region <= conn_threshold)
        regions = regionprops(labeled_region)

        # 计算感兴趣区域的大小
        regions = [region for region in regions if region.area >= parameters['min_area']]
        num_region = len(regions)

        if parameters['verbose']:
            print(f"Number of regions: {num_region}")

        if num_region == 0:
            if conn_pointer < len(conn_ratio):
                conn_pointer += 1
            else:
                break
        # 2-概率基元行进
        # 针对每个区域寻找最佳的超四面体表示
        num_region = len(regions)
        x_temp = np.zeros((num_region, 11))
        del_idx = np.zeros(num_region, dtype=int)
        occ_idx_in = [None] * num_region
        num_idx = np.zeros((num_region, 3))
        
        # 创建一个字典来存储每个区域的像素索引
        region_pixel_idx_lists = {region.label: np.where(labeled_region == region.label) for region in regions}

        for i in range(num_region):
            # 获取并调整边界框
            bbox = regions[i].bbox
            bbox = np.ceil(bbox).astype(int)
            bbox[3:] = np.minimum(bbox[:3] + bbox[3:] + parameters['padding_size'], 
                                [voxel_grid['size'][1], voxel_grid['size'][0], voxel_grid['size'][2]])
            bbox[:3] = np.maximum(bbox[:3] - parameters['padding_size'], 1)

            # 计算激活的体素索引
            idx_x, idx_y, idx_z = np.mgrid[bbox[1]:bbox[4], bbox[0]:bbox[3], bbox[2]:bbox[5]]
            indices = np.vstack([idx_x.ravel(), idx_y.ravel(), idx_z.ravel()]).T
            roi_idx = idx3d_flatten(indices, voxel_grid)
            regions[i].idx = roi_idx

            # 计算边界点坐标
            bounding_points = idx2Coordinate(np.array([
                [bbox[1], bbox[1], bbox[4], bbox[4], bbox[1], bbox[1], bbox[4], bbox[4]],
                [bbox[0], bbox[0], bbox[0], bbox[0], bbox[3], bbox[3], bbox[3], bbox[3]],
                [bbox[2], bbox[5], bbox[2], bbox[5], bbox[2], bbox[5], bbox[2], bbox[5]]
            ]), voxel_grid)
            regions[i].bounding_points = bounding_points
            # 确定中心点并向下取整
            centroid = np.maximum(np.floor(regions[i].centroid), 1)

            # 将中心点坐标转换为一维索引
            centroid_flatten = idx3d_flatten(np.array([[centroid[1], centroid[0], centroid[2]]]), voxel_grid)

            # 检查一维索引是否在ROI的像素索引列表中
            pixel_idx_list = region_pixel_idx_lists[regions[i].label]
            # 将像素索引列表转换为一个二维数组
            pixel_idx_array = np.vstack(pixel_idx_list).T

            # 检查 centroid_flatten 是否在 pixel_idx_array 中
            if np.any(np.all(centroid_flatten == pixel_idx_array, axis=1)):
            # if centroid_flatten in region_pixel_idx_lists[regions[i].label]:
                regions[i].centroid = voxel_grid['points'][:, centroid_flatten[0]]
            else:
                print(pixel_idx_list)
                pixel_coords = voxel_grid['points'][:, pixel_idx_list].T
                # 最近邻搜索找到最接近的点
                kdtree = cKDTree(pixel_coords)
                _, nearest_point_idx = kdtree.query(voxel_grid['points'][:, centroid_flatten].T)
                regions[i].centroid = voxel_grid['points'][:, regions[i].pixel_idx_list[nearest_point_idx]]


        # 更新划分深度
        num_division += 1

    return x

def connectivity_check(sdf, grid, threshold):
    """
    检查和标记连接区域。
    返回标记的区域和每个区域的属性（如中心点、面积等）。
    """
    # 这里可以使用skimage的label和regionprops函数
    # ...

def probabilistic_primitive_marching(sdf, grid, parameters):
    """
    在每个划分的区域中寻找最优的超四面体表示。
    这可能涉及复杂的几何计算和优化算法。
    """
    # ...

def optimize_superquadric(sdf, initial_guess, grid, parameters):
    """
    使用非线性最小二乘法或其他优化方法来调整超四面体参数，以便最佳地拟合给定的SDF。
    """
    # ...

In [67]:

# 使用tkinter获取文件路径
root = tk.Tk()
root.withdraw() # 防止Tkinter窗口出现
file_path = r"D:\Marching-Primitives-Python\data\chair1_normalized.csv"
if not file_path:
    raise ValueError("No file selected.")

# 读取CSV文件
sdf = np.genfromtxt(file_path, delimiter=',').T
voxelGrid = {}

# 设置体素网格参数
voxelGrid['size'] = np.ones(3, dtype=int) * int(sdf[0])
voxelGrid['range'] = sdf[1:7]
sdf = sdf[7:]
# 创建线性空间
voxelGrid['x'] = np.linspace(voxelGrid['range'][0], voxelGrid['range'][1], int(voxelGrid['size'][0]))
voxelGrid['y'] = np.linspace(voxelGrid['range'][2], voxelGrid['range'][3], int(voxelGrid['size'][1]))
voxelGrid['z'] = np.linspace(voxelGrid['range'][4], voxelGrid['range'][5], int(voxelGrid['size'][2]))

# 创建网格
x, y, z = np.meshgrid(voxelGrid['x'], voxelGrid['y'], voxelGrid['z'], indexing='ij')
voxelGrid['points'] = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T

# 计算间隔和截断
voxelGrid['interval'] = (voxelGrid['range'][1] - voxelGrid['range'][0]) / (voxelGrid['size'][0] - 1)
voxelGrid['truncation'] = 1.2 * voxelGrid['interval']
voxelGrid['disp_range'] = [-np.inf, voxelGrid['truncation']]
voxelGrid['visualizeArclength'] = 0.01 * np.sqrt(voxelGrid['range'][1] - voxelGrid['range'][0])

# 截断SDF
sdf = np.clip(sdf, -voxelGrid['truncation'], voxelGrid['truncation'])
print('sdf.shape: ', sdf.shape)
print('voxelGrid[\'points\'].shape: ', voxelGrid['points'].shape)



sdf.shape:  (125000,)
voxelGrid['points'].shape:  (125000, 3)


In [68]:
# %% marching-primitives
# Python中的时间测量
import time
start_time = time.time()
# 这里你需要用Python实现或找到一个库来实现MPS算法
x = mps(sdf, voxelGrid) # 假设MPS是已实现的函数
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time} seconds")


Number of regions: 1
(array([14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18,
       18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21,
       21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
       21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
       22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23,
   

IndexError: index 14 is out of bounds for axis 1 with size 3

In [ ]:

# %% triangularization and compression
# 你需要找到或实现一个类似于meshSuperquadrics的函数
# mesh_original = meshSuperquadrics(x, 'Arclength', voxelGrid['visualizeArclength'])
# 压缩 - 使用skimage.measure的approximate_polygon或类似功能
# mesh = reducepatch(mesh_original) # reducepatch需要实现或找到替代方案

# TODO: 实现STL写入 - 可能需要一个库或自定义函数

# 判断是否保存
ifsave = True
if ifsave:
    x_save = x.astype(np.float32)
    # 保存x_save和stl文件
    np.save(os.path.join(pathname, f"{name}_sq.npy"), x_save)
    # stlwrite(stl, os.path.join(pathname, f"{name}_sq.stl")) # 需要实现stlwrite

# %% visualize
# 关闭所有图形 - 在Python中，你可以使用plt.close('all')
# 设置视图参数和颜色

# 可视化部分需要重写以适应Python的matplotlib或类似库
# 使用plt.trisurf等函数进行绘制

# 注意：由于Python和MATLAB在图形处理和特定函数实现上有显著差异，这部分代码需要根据你的具体需求进行调整。

NameError: name 'pathname' is not defined